In [2]:
import os
from pydub import AudioSegment
import numpy as np

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, LSTM, Bidirectional, Input, concatenate, Dropout, SpatialDropout1D, Flatten, BatchNormalization, Conv1D, MaxPooling1D
from keras.utils.np_utils import to_categorical
from keras import regularizers
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

## Raw data preprocessing:

In [5]:
MAP_DIR_PATH = 'MAPS/PROTOTYPE MAPS/'
MAP_NAME = 'combined_maps.txt'

In [6]:
# Load the beatmap hitcirlces

with open(os.path.join(MAP_DIR_PATH, MAP_NAME + '.osu')) as f:
    content = [x.strip() for x in f.readlines()]

hitobjects_index = [x for x in content].index('[HitObjects]')
hitobjects = [x.split(',')[:3] for x in content[hitobjects_index + 1:]]

for i in range(10):
    print(hitobjects[i])

80,64,107,1,0
128,192,380,5,0
160,192,516,1,0
176,320,789,1,0
208,320,925,1,0
232,192,1198,1,0
264,192,1334,1,0
288,320,1607,1,0
320,320,1743,1,0
336,192,2016,1,0


In [7]:
# Preprocesses the labels

print(len(hitobjects))
# X and y cordinate labels
time_values = []
y_x = []
y_y = []

map_start = int(hitobjects[0][-1])

for y in hitobjects:
    y_x.append(y[0])
    y_y.append(y[1])
    time_values.append(int(y[-1]))
    
print('Time_values:', time_values[1], 'x:', y_x[1], 'y:', y_y[1])

NameError: name 'hitobjects' is not defined

In [5]:
# Load the song

map_song = AudioSegment.from_file(os.path.join(MAP_DIR_PATH, MAP_NAME + '.mp3'), format='mp3')

print('Length before: ', len(map_song))
song_in_map_range = map_song[map_start:int(hitobjects[-1][-1])]
print('Length after: ', len(song_in_map_range))

print('Length of samples in one timestep: ', len(song_in_map_range[0].get_array_of_samples()))

Length before:  100127
Length after:  78405
Length of samples in one timestep:  88


In [9]:
OUTPUTS_PER_SECOND = 20
INPUTS_PER_TIMESTEP = 250

# Loop over each second of the song and get the data and labels for timesteps
# loop over the song ms by ms, if ms has hitcircle on it, append 1(coordinates included) else 0 (coordinates not included)
X = []
y = []
timing_matches = 0
for i in range(0,  len(song_in_map_range)):
    X.append(np.asarray(song_in_map_range[i].get_array_of_samples()))
    if i in time_values:
        y.append([y_x[timing_matches], y_y[timing_matches]])
        timing_matches += 1
    else:
        y.append(['nan', 'nan'])
    

        
print(y[0])
print(y[1])
print('Total timing matches', len([i for i in y if i[0] != 'nan']), 'out of', len(y))
print('Len of X:', len(X), '\nexample values:', X[0][0:5])

print('\n\nX and y pairs:', '\nX:', X[0][0:5], '\ny:', y[0])

['nan', 'nan']
['nan', 'nan']
Total timing matches 156 out of 78405
Len of X: 78405 
example values: [ -174  9910  2588 10408  5989]


X and y pairs: 
X: [ -174  9910  2588 10408  5989] 
y: ['nan', 'nan']


In [7]:
# Loop over the hitcircles and get audio data from 1 second before and 1 second after the hitcirclec 
# loop over the song ms by ms, if ms has hitcircle on it, append 1(coordinates included) else 0 (coordinates not included)
X = []
timing_matches = 0

for i, y in enumerate(time_values):
    X.append([])
    for j in range(-1000, 0):        
        X[i].append(map_song[y + j].get_array_of_samples().tolist())
        
    for j in range(1000):
        X[i].append(map_song[y + j].get_array_of_samples().tolist())
        

print('Len of X:', len(X), '\namount of examples:', len(X[0]))

Len of X: 233 
amount of examples: 2000


In [10]:
# Reduce X dimensionality
array_min = min(X[0])
array_max = max(X[0])

maxval = 5000 # HYPERPARAMETER to reduce dimensionality

print('Array min:',array_min, 'array max:',array_max)

divider = array_max / maxval
print('Divider:', divider)

print('Before:', X[0][0:10])
print('After:',X[0][0:10] // divider)

print('Max:',max(X[0] // divider))
print('Min:',min(X[0] // divider))

X = X // divider

Array min: -17411 array max: 18648
Divider: 3.7296
Before: [ -174  9910  2588 10408  5989 13656  7966 14921  3180  8682]
After: [ -47. 2657.  693. 2790. 1605. 3661. 2135. 4000.  852. 2327.]
Max: 4999.0
Min: -4669.0


## Preparing data for the classifier:

Turn coordinate values into embeddings

In [19]:
from sklearn.model_selection import train_test_split

stringified_X = []
for ind, x in enumerate(X):
    listed_str = []
    
    for i in x:        
        listed_str.append(str(int(i)))
    stringified_X.append(' '.join(listed_str))

stringified_y = [''.join(i) for i in y] # should be ' '.join

X_train, X_val, y_train, y_val = train_test_split(stringified_X, stringified_y, test_size=0.25, shuffle=False)

# turn coordinate values into a big string
print(X_train[0][0:100])
print(X_val[0][0:100])

print(y_train[0])
print(y_val[0])

-47 2657 693 2790 1605 3661 2135 4000 852 2327 -1193 504 -554 1387 2179 3177 2645 1942 199 -1511 -17
-823 -815 -798 -554 -573 -408 -217 -275 25 -185 105 -467 382 -742 676 -790 756 -839 902 -729 786 -61
nannan
nannan


In [20]:
label_vocab_size = 1500 # Don't really need more than 1000, but for some reason a bigger vocab_size is good for the tokenizer
label_seq_len = 2

feature_vocab_size = 15000
feature_seq_len = 88

def tokenization_processing(x_to_tok, x_val_to_tok, vocab_size=1500, seq_len=2, pad=False):

    tokenizer = Tokenizer(num_words=vocab_size,
                                       filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                       split=" ",
                                       char_level=False, # Could be very very interesting
                                       oov_token=None)

    tokenizer.fit_on_texts(x_val_to_tok + x_to_tok)
    print('True vocab size:', len(tokenizer.word_index))
    
    X_tokenized = tokenizer.texts_to_sequences(x_to_tok)
    X_val_tokenized = tokenizer.texts_to_sequences(x_val_to_tok)
    
    if pad:
        X_tokenized = pad_sequences(X_tokenized, maxlen=seq_len)
        X_val_tokenized = pad_sequences(X_val_tokenized, maxlen=seq_len)
    
    print('Before:', x_val_to_tok[0])
    print('After:', X_val_tokenized[0])
    print('Before:', x_to_tok[0])
    print('After:',X_tokenized[0])
    
    return [X_tokenized, X_val_tokenized, tokenizer]

Y_tokenized, y_val_tokenized, tokenizer_y = tokenization_processing(y_train, y_val)
X_train_tokenized, X_val_tokenized, tokenizer_x = tokenization_processing(X_train, X_val, vocab_size=feature_vocab_size, seq_len=feature_seq_len, pad=True)

True vocab size: 128
Before: nannan
After: [1]
Before: nannan
After: [1]
True vocab size: 8786
Before: -823 -815 -798 -554 -573 -408 -217 -275 25 -185 105 -467 382 -742 676 -790 756 -839 902 -729 786 -610 109 -731 -337 -582 -224 -183 -153 -207 -263 -422 -254 -387 -112 -447 -39 -737 -112 -909 -105 -872 118 -697 223 -451 39 -269 -40 -57 163 247 324 345 415 230 683 162 1041 118 1277 197 1258 396 941 338 688 245 685 482 489 544 42 236 -172 75 -42 151 243 252 497 245 621 96 818 102 1065 236
After: [ 608  545  972  718  758  255   56   95  412  563  128  243  217  670
  761  439  483 1082 1088  883  555  751  111  361  582  262  130  119
  120  417  527  667   31  166  108  479   52  510  108  992  128 1112
  458  485   75  700   52  468  427   78  392  517  182   83  601  552
  475  543  769  458 1107  385 1380  386  733   91  940  441  804  219
  661  710   29  440   27   73   29  532  160  398  190  441  825  461
  936    8 1309  440]
Before: -47 2657 693 2790 1605 3661 2135 4000 852 2327

In [21]:
print(len(Y_tokenized))
print(len(X_train_tokenized))


def create_mini_batches(input_X, input_y, batch_size):
    batched_X = []
    batched_y = []
    for i in range(int(len(input_X) / batch_size) + 1):
        batched_X.append(input_X[i*batch_size:i*batch_size + batch_size])
        batched_y.append(input_y[i*batch_size:i*batch_size + batch_size])

    return (batched_X, batched_y)

X_train_timestepped, y_train_timestepped = create_mini_batches(X_train_tokenized, Y_tokenized, 256)
X_val_timestepped, y_val_timestepped = create_mini_batches(X_val_tokenized, y_val_tokenized, 256)

print(len(X_train_timestepped))
print(len(X_val_timestepped))

58803
58803
230
77


In [24]:
# define model
model = Sequential()
model.add(Embedding(feature_vocab_size, 10, input_length=feature_seq_len))
model.add(LSTM(50))
#model.add(LSTM(50))
#model.add(Dense(256, activation='relu'))
#model.add(Dense(256, activation='relu'))
#model.add(Dense(256, activation='relu'))
model.add(Dense(label_vocab_size, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 88, 10)            150000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)                12200     
_________________________________________________________________
dense_5 (Dense)              (None, 1500)              76500     
Total params: 238,700
Trainable params: 238,700
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
#model.fit(np.asarray(X_train_timestepped), np.asarray(y_train_timestepped), epochs=5, verbose=1, batch_size=32, validation_data=(np.asarray(X_val_timestepped), np.asarray(y_val_timestepped)))
class_weight = {y: 1. if y != 1 else 0.000001 for y in range(200)}
print(class_weight)
model.fit(np.asarray(X_train_tokenized), np.asarray(Y_tokenized), class_weight=class_weight, epochs=100, verbose=1, batch_size=32, validation_data=(np.asarray(X_val_tokenized), np.asarray(y_val_tokenized)))

{0: 1.0, 1: 1e-06, 2: 1.0, 3: 1.0, 4: 1.0, 5: 1.0, 6: 1.0, 7: 1.0, 8: 1.0, 9: 1.0, 10: 1.0, 11: 1.0, 12: 1.0, 13: 1.0, 14: 1.0, 15: 1.0, 16: 1.0, 17: 1.0, 18: 1.0, 19: 1.0, 20: 1.0, 21: 1.0, 22: 1.0, 23: 1.0, 24: 1.0, 25: 1.0, 26: 1.0, 27: 1.0, 28: 1.0, 29: 1.0, 30: 1.0, 31: 1.0, 32: 1.0, 33: 1.0, 34: 1.0, 35: 1.0, 36: 1.0, 37: 1.0, 38: 1.0, 39: 1.0, 40: 1.0, 41: 1.0, 42: 1.0, 43: 1.0, 44: 1.0, 45: 1.0, 46: 1.0, 47: 1.0, 48: 1.0, 49: 1.0, 50: 1.0, 51: 1.0, 52: 1.0, 53: 1.0, 54: 1.0, 55: 1.0, 56: 1.0, 57: 1.0, 58: 1.0, 59: 1.0, 60: 1.0, 61: 1.0, 62: 1.0, 63: 1.0, 64: 1.0, 65: 1.0, 66: 1.0, 67: 1.0, 68: 1.0, 69: 1.0, 70: 1.0, 71: 1.0, 72: 1.0, 73: 1.0, 74: 1.0, 75: 1.0, 76: 1.0, 77: 1.0, 78: 1.0, 79: 1.0, 80: 1.0, 81: 1.0, 82: 1.0, 83: 1.0, 84: 1.0, 85: 1.0, 86: 1.0, 87: 1.0, 88: 1.0, 89: 1.0, 90: 1.0, 91: 1.0, 92: 1.0, 93: 1.0, 94: 1.0, 95: 1.0, 96: 1.0, 97: 1.0, 98: 1.0, 99: 1.0, 100: 1.0, 101: 1.0, 102: 1.0, 103: 1.0, 104: 1.0, 105: 1.0, 106: 1.0, 107: 1.0, 108: 1.0, 109: 1.0, 110: 1.

KeyboardInterrupt: 

In [42]:
model.save('MODEL/33_epochs_music.h5')
model.save_weights('MODEL/33_epochs_music_weights.h5')
np.save('MODEL/33_epochs_music_weights.np', model.get_weights())

In [35]:
def generate_seq(model, tokenizer, max_length, seed_text, n_words, sequence_len=feature_seq_len):
    end_text = seed_text
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded = np.array(tokenizer.texts_to_sequences([in_text]))        
        if(encoded.shape != (1, sequence_len)):
            encoded = np.array([np.append(encoded[0], 0)])
        # pre-pad sequences to a fixed length
        #encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
        # predict probabilities for each word
        yhat = model.predict_classes(encoded, verbose=0)
        print(yhat)     
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
            elif yhat==[0]:
                out_word = '.'
                end_text += ' ' + out_word
                return end_text
        # append to input
        end_text += ' ' + out_word
        in_text = in_text.split(' ')[1:]
        in_text.append(out_word)
        in_text = ' '.join(in_text)        
        
    return end_text

print(generate_seq(model, tokenizer_x, feature_seq_len-1, '-823 -815 -798 -554 -573 -408 -217 -275 25 -185 105 -467 382 -742 676 -790 756 -839 902 -729 786 -610 109 -731 -337 -582 -224 -183 -153 -207 -263 -422 -254 -387 -112 -447 -39 -737 -112 -909 -105 -872 118 -697 223 -451 39 -269 -40 -57 163 247 324 345 415 230 683 162 1041 118 1277 197 1258 396 941 338 688 245 685 482 489 544 42 236 -172 75 -42 151 243 252 497 245 621 96 818 102 1065 236', 10, sequence_len=feature_seq_len))

[1]
[16]
[83]
[18]
[105]
[105]
[105]
[105]
[16]
[110]
-823 -815 -798 -554 -573 -408 -217 -275 25 -185 105 -467 382 -742 676 -790 756 -839 902 -729 786 -610 109 -731 -337 -582 -224 -183 -153 -207 -263 -422 -254 -387 -112 -447 -39 -737 -112 -909 -105 -872 118 -697 223 -451 39 -269 -40 -57 163 247 324 345 415 230 683 162 1041 118 1277 197 1258 396 941 338 688 245 685 482 489 544 42 236 -172 75 -42 151 243 252 497 245 621 96 818 102 1065 236 135 158 345 94 2 2 2 2 158 6


In [40]:
print(list(tokenizer_y.word_index.items())[1])
print(list(tokenizer_y.word_index.items())[16])
print(list(tokenizer_y.word_index.items())[83])
print(list(tokenizer_y.word_index.items())[18])
print(list(tokenizer_y.word_index.items())[105])
print(list(tokenizer_y.word_index.items())[105])
print(list(tokenizer_y.word_index.items())[105])
print(list(tokenizer_y.word_index.items())[105])
print(list(tokenizer_y.word_index.items())[16])
print(list(tokenizer_y.word_index.items())[110])

('256192', 2)
('256344', 17)
('216272', 84)
('192243', 19)
('352240', 106)
('352240', 106)
('352240', 106)
('352240', 106)
('256344', 17)
('25648', 111)


In [7]:
# Preprocess the audio data

OUTPUTS_PER_SECOND = 20
INPUTS_PER_TIMESTEP = 250

# Loop over each second of the song and get the audio data for timesteps
X = []
for ind, i in enumerate(range(0,  len(song_in_map_range), 1000)):
    X.append([])
    
    # Step in timestep frequency, eg 4
    for j in range(0, len(song_in_map_range[i:i+1000]), 1000//INPUTS_PER_TIMESTEP):
        X[ind].append(song_in_map_range[j].get_array_of_samples())
        
    # This doesn't work because of the slicing step:
    #X.append(song_in_map_range[i:i+1000:1000//INPUTS_PER_TIMESTEP])            
    

print('Length of the song (X): ', len(X))
print('Length of one second: ', len(X[0]))

Length of the song (X):  79
Length of one second:  250
